# Web Scraping
![Web Scrape](img/web-scrape.png)
* 網站使用 HTML 做為預設的文件格式, 由於 HTML 有規範與標準, 瀏覽器可以依據規範來顯示畫面
* 有時候 HTML 文件沒有依照規範來編排, 這種狀況被稱為 tag soup
* 使用 requests 模組擷取 HTML 文件, 再透過 BeautifulSoup 模組分析 tag 內容, 是常見的處理方式

# requests 模組
* 存取網站檔案的優秀工具
* 網頁檔案常見的存取方式是 Get 和 Post 模式, Get 模式的參數可以從網址看到, Post 模式不會把參數在網址顯示
* 正常擷取網頁的話, 回傳值是 200, 其他號碼可參考 [HTTP Status Code](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes) 文件說明

In [ ]:
# 載入模組
import requests

In [ ]:
# 回傳 Response 物件
req = requests.get("https://api.github.com/events")

In [ ]:
# 檢查 HTTP Status Code 回傳值
req

In [ ]:
# 顯示網頁內容
req.text

In [ ]:
# 顯示內容的編碼格式, 預設應是 utf-8
req.encoding

In [ ]:
# 在網址輸入參數
payload = {'key1': 'value1', 'key2': 'value2'}
req2 = requests.get('http://httpbin.org/get', params=payload)

# 註: httpbin.org 是一個適合測試練習 HTTP 服務的網站

In [ ]:
print(req.url)

In [ ]:
# 參數可以是 List of Strings
payload = {'key1': 'value1', 'key2': ['value2', 'value3']}
req2 = requests.get('http://httpbin.org/get', params=payload)

In [ ]:
print(req.url)

In [ ]:
# Post 範例
req3 = requests.post('http://httpbin.org/post', data = {'key': 'value'})

## 常見錯誤訊息
* SSLError: ("bad handshake: Error([('SSL routines', 'SSL23_GET_SERVER_HELLO', 'tlsv1 unrecognized name')],)",) -- Mac User 可能要[檢查 OpenSSL 安裝狀況](https://github.com/requests/requests/issues/2022)
* Response [403]: Forbidden -- 可能要求明確的 User-Agent 參數
* TypeError: object of type 'Response' has no len() -- 要使用 req.text
* UnicodeEncodeError: 'cp950' codec can't encode character '\xa0' in position 3999: illegal multibyte sequence
* [big5 encoding 處理方式](http://sjkou.net/2017/01/06/python-requests-encoding/)

# BeautifulSoup 模組
* 用來分析存取 HTML 文件的工具, 會將 HTML 文件轉換成 Python Object Tree
* 常見的 Object 型別是 Tag
* 輸入的資料預設會使用 Unicode 編碼, 輸出的資料預設使用 UTF-8 編碼

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
# 範例: 最近中文新書清單
url = "http://www.tenlong.com.tw/zh_tw/recent"
tenlong = requests.get(url)

In [ ]:
# 建立 BeautifulSoup 物件, 參數指定 'html.parser'
soup = BeautifulSoup(tenlong.text, 'html.parser')

In [ ]:
# 顯示排版後的 HTML 內容
soup.prettify()

In [ ]:
# 顯示 Soup 第一階 Object Tree 物件名稱
list(soup.children)

In [ ]:
# 顯示 Soup 第一階物件的資料型別
for item in soup.children:
    print(type(item))

# Tag Object 是 BeautifulSoup 最常被使用的資料型別

In [ ]:
# 顯示所有文字內容
soup.get_text()

# 認識物件樹
![Object Tree](img/object-tree.png)
* 最常見是利用 .contents 和 .children 屬性來向下查詢, 如果 Children 裡更底層的 Children 都要存取, 可用 .descendants 屬性
* .contents 回傳 List, .children 回傳 Generator

# Tag Object
* Tag 之間有 Children, Parent, Sibling 的關係
* 可以搭配瀏覽器"檢查元素"功能, 找出需要的 Tag 規則
* 常見技巧: 顯示所有 Link 和文字內容

In [ ]:
# 顯示所有 Link 網址
for link in soup.find_all('a'):
    print(link.get('href'))

In [ ]:
# 顯示 p Tag 的文字內容
p.get_text()

# 利用 find_all() 找出符合條件的資料
* 格式: find_all("tag_name", arguments)

In [ ]:
# 找出所有 li (List) Tag 的內容
soup.find_all("li")

In [ ]:
# 指定 class 參數
soup.find_all("li", {"class": "single-book"})

In [ ]:
# 找出所有新書 title
book_list = soup.find_all("li", {"class": "single-book"})
for book in book_list:
    print(book.h3.get_text())

# 如果有額外的空白, 可用 strip() 濾除

In [ ]:
# 找出所有新書 pricing
for book in book_list:
    print(list(book.div.children)[-1])

In [ ]:
# 找出所有新書 product URL
for book in book_list:
    print(book.h3.a['href'])

# 可能需要補上 prefix 成為完整網址

# 利用檔案記錄需要的資料
* CSV 是常見的檔案格式, 可用 , 或 \t 來分隔欄位
* 中文內容的檔案, 如果沒有指定 encoding="utf-8" 可能會有亂碼問題

In [ ]:
import requests
from bs4 import BeautifulSoup

tenlong = requests.get("http://www.tenlong.com.tw/zh_tw/recent")
soup = BeautifulSoup(tenlong.text, 'html.parser')

book_list = soup.find_all("li", {"class": "single-book"})
lines = []
for book in book_list:
    title = book.h3.get_text().strip()
    price = list(book.div.children)[-1].replace('售價:','').strip()
    p_url = book.h3.a['href']
    lines.append(title + '\t' + price + '\t' + p_url + '\n')
with open("tenlong-recent.csv", "w", encoding="utf-8") as f:
    f.writelines('Title\tPrice\tProduct_URL\n')
    f.writelines(lines)

# 換頁擷取網頁
* 試著找看看網址規則

In [ ]:
soup.find_all("a", {"class": "next_page"})

# 指定 User Agent String
* 有些網址要求明確的 User Agent 資料, 不然無法透過 requests 存取
* 範例網址: http://packtpub.com/packt/offers/free-learning
* 利用 Google 查詢 User Agent: what is my user agent string

In [ ]:
# 試試直接要求能否成功?
url = "http://packtpub.com/packt/offers/free-learning"
packt = requests.get(url)
packt

In [ ]:
# 透過 Dictionary 指定 User Agent 資料
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:57.0) Gecko/20100101 Firefox/57.0'}

In [ ]:
# 加上 headers 參數
packt = requests.get(url, headers=headers)

# 輸入 Search Keyword 取得查詢結果
* 範例網址: http://packtpub.com/all

In [ ]:
# 請配合自己的需要修改參數值
url = 'http://packtpub.com/all'
params = {'search': 'python'}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:57.0) Gecko/20100101 Firefox/57.0'}

In [ ]:
packt = requests.get(url, params=params, headers=headers)

In [ ]:
soup = BeautifulSoup(packt.text, 'html.parser')

# CSS Selector
* 使用 select() Method

In [ ]:
soup.select("div p")

# API
* [YouTube Comment](https://developers.google.com/youtube/v3/docs/comments)

# 使用 Selenium 模組
* HTML 是經由 [JavaScript render](http://pala.tw/python-web-crawler/) 產生時, 就得搭配 [Selenium](http://medium.com/@hoppy/how-to-test-or-scrape-javascript-rendered-websites-with-python-selenium-a-beginner-step-by-c137892216aa) 之類的模組來爬取內容
* 先確認是否已安裝 -- [在 Anaconda 環境安裝](https://stackoverflow.com/questions/46137219/how-to-install-selenium-in-a-conda-environment)
* conda install -c conda-forge selenium

In [ ]:
import selenium

browser = selenium.webdriver.Chrome()
browser.get("http://anaconda.org/")
nav = browser.find_element_by_css_selector('ul.right')

# Regular Expression
* 中文稱為正規表示式或正則表達式, 英文簡寫成 regex, 常用於字串比對, 例如找出文件裡的電話號碼/身份證號碼/電子郵件地址/網址
* 建議使用 [Raw String](http://regexone.com/references/python) 來簡化 \ 符號的影響
![Email Example](https://www.computerhope.com/jargon/r/regular-expression.gif)

In [ ]:
import re
re.findall(r'\S+', re.sub(r'([?.])', r' \1', 'What are you doing?'))